In [6]:
import json
from anytree import Node, RenderTree
from anytree.search import find

class TaxonomyParser:

    """
    This class is a wrapper on a Tree class from the anytree library to hold
    different data hierarchies read from a JSON representation
    """

    def __init__(self, input_file=None):
        self.nodes = {"Root": Node("Root", parent=None, level=-1)}
        self.taxo_mappings = {}
        self.root_key = None
        self.input_file = input_file
        # TODO check if file exists
        self.read_from_json()

    def read_from_json(self):
        """
        Read the taxonomy from a JSON file given as input
        """
        try:
            with open(self.input_file, "r") as f:
                data = json.load(f)
                n_levels = len(list(data.keys()))

                 # populate the tree
                for key, description in data.items():
                    self.taxo_mappings[key] = description
                    # description = arts, culture and entertainment
                    # dewcription = arts, culture and entertainment>animation
                    categories = description.split(">")
                
                    for level, category_name in enumerate(categories):
                        if level==0 and category_name not in self.nodes:
                            # if top-level category and doesn't exist yet, then create Node with Parent root
                            self.nodes[category_name] = Node(category_name, parent=self.nodes["Root"], level=level)
                        elif category_name not in self.nodes:
                            # not a top-level category, create it with the proper Parent
                            self.nodes[category_name] = Node(category_name, parent=self.nodes[last_name], level=level)
                        # remember the current node
                        last_name = category_name
        except Exception as e:
            print(e)
   

if __name__ == "__main__":
    d = TaxonomyParser(input_file="taxonomy_mappings.json")

In [7]:
print(RenderTree(d.nodes["Root"]))

Node('/Root', level=-1)
├── Node('/Root/arts, culture and entertainment', level=0)
│   ├── Node('/Root/arts, culture and entertainment/animation', level=1)
│   ├── Node('/Root/arts, culture and entertainment/art exhibition', level=1)
│   ├── Node('/Root/arts, culture and entertainment/bullfighting', level=1)
│   ├── Node('/Root/arts, culture and entertainment/cartoon', level=1)
│   ├── Node('/Root/arts, culture and entertainment/cinema', level=1)
│   ├── Node('/Root/arts, culture and entertainment/comedy events', level=1)
│   ├── Node('/Root/arts, culture and entertainment/cultural development', level=1)
│   ├── Node('/Root/arts, culture and entertainment/customs and tradition', level=1)
│   ├── Node('/Root/arts, culture and entertainment/entertainment award', level=1)
│   ├── Node('/Root/arts, culture and entertainment/fan conventions', level=1)
│   ├── Node('/Root/arts, culture and entertainment/festive event', level=1)
│   ├── Node('/Root/arts, culture and entertainment/fine art', l

# Read Trainings Data

In [ ]:
class Document():
    def __init__(publishedAt, title, content, sections, labels ):
        self.metadata = 


class TextCorpus():
    def __init__(self, fname=None, taxo_map=None):
        self.taxo_tree = TaxonomyParser(input_file=taxo_map)
        self.input_file = fname
        # check if fname exists
        
        
    def read_data()
    